In [1]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.7 MB/s eta 0:00:00


In [2]:

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd


In [3]:
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
OUTPUT_DIR = "./phi3-full-resume-enhanced"
MAX_LENGTH = 2048  # Increased for full resumes


In [4]:
def create_full_resume_dataset():
    """Create dataset with full resume examples"""

    data = [
        {
            "input": """JOHN DOE - RESUME

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
Location: New York, NY

PROFESSIONAL SUMMARY:
Software Developer with 3 years of experience in web development.
Skilled in Python, JavaScript, and building responsive applications.

WORK EXPERIENCE:
- Junior Developer at Tech Solutions Inc. (2022-Present)
- Developed web applications using Python and Flask
- Created responsive front-end designs with HTML/CSS

EDUCATION:
Bachelor of Computer Science
University of Technology, 2021

SKILLS:
- Programming: Python, JavaScript, HTML, CSS
- Tools: Git, SQL""",
            "output": """JOHN DOE
New York, NY | (555) 123-4567 | john.doe@email.com | john.doe@email.com

PROFESSIONAL SUMMARY
Highly skilled Software Developer with 3 years of experience in web development, proficient in Python, JavaScript, and building responsive applications.

WORK EXPERIENCE
JUNIOR DEVELOPER
Tech Solutions Inc., New York, NY (2022-Present)
- Developed and maintained web applications using Python and Flask
- Designed and implemented responsive front-end solutions with HTML/CSS

EDUCATION
BACHELOR OF COMPUTER SCIENCE
University of Technology, New York, NY (2017-2021)

SKILLS
- Programming: Python, JavaScript, HTML, CSS
- Tools: Git, VS Code, PowerShell
- Languages: English (Fluent)"""
        },
        {
            "input": """Sarah Chen
sarah.chen@email.com
(555) 987-6543

Summary:
Marketing professional with experience in social media and content creation.

Experience:
Marketing Coordinator at Digital Agency (2021-2023)
- Managed social media accounts
- Created content for various platforms
- Worked with design team

Education:
BA in Marketing, State University, 2021

Skills: Social media, Content writing, Adobe Suite""",
            "output": """SARAH CHEN
Location | (555) 987-6543 | sarah.chen@email.com

PROFESSIONAL SUMMARY
Results-driven Marketing Professional with 2+ years of experience specializing in social media management, content creation, and cross-functional collaboration. Demonstrated expertise in digital marketing strategies and creative content development.

WORK EXPERIENCE
MARKETING COORDINATOR
Digital Agency (2021-2023)
- Orchestrated multi-platform social media strategy across 5+ channels, growing engagement by 45% through targeted content campaigns
- Developed and executed 50+ high-impact content pieces monthly, resulting in 30% increase in audience reach
- Collaborated with cross-functional design team to deliver 100+ branded assets, ensuring cohesive brand messaging across all touchpoints

EDUCATION
BACHELOR OF ARTS IN MARKETING
State University (2017-2021)

SKILLS
- Digital Marketing: Social Media Strategy, Content Marketing, Campaign Management
- Creative Tools: Adobe Creative Suite (Photoshop, Illustrator, InDesign)
- Content Creation: Copywriting, Brand Storytelling, Visual Content Development
- Analytics: Social Media Analytics, Performance Tracking"""
        },
        {
            "input": """Mike Johnson
mike.j@email.com | 555-234-5678

About Me:
Data analyst who likes working with numbers and Excel.

Work:
- Data Analyst at Finance Corp (2020-Present)
- Made reports and charts
- Used Excel a lot

School:
BS Mathematics, Tech College, 2020

What I know:
Excel, SQL, some Python""",
            "output": """MIKE JOHNSON
Location | (555) 234-5678 | mike.j@email.com

PROFESSIONAL SUMMARY
Analytical Data Analyst with 3+ years of experience transforming complex datasets into actionable business insights. Specialized in advanced data visualization, statistical analysis, and leveraging Python and SQL for data-driven decision making.

WORK EXPERIENCE
DATA ANALYST
Finance Corp (2020-Present)
- Engineered 40+ comprehensive financial reports and interactive dashboards using Excel and Power BI, enabling executive-level data-driven decisions that improved operational efficiency by 25%
- Conducted advanced statistical analysis on datasets exceeding 500K records, identifying key trends that generated $300K in cost savings annually
- Automated data processing workflows using Python scripts, reducing manual reporting time by 60% and improving data accuracy by 95%

EDUCATION
BACHELOR OF SCIENCE IN MATHEMATICS
Tech College (2016-2020)

SKILLS
- Data Analysis: Statistical Analysis, Predictive Modeling, Data Mining
- Technical Tools: Advanced Excel (Pivot Tables, Macros, VBA), SQL, Python (Pandas, NumPy)
- Visualization: Power BI, Tableau, Excel Charts
- Database: SQL Server, MySQL, Data Querying"""
        },
        {
            "input": """Emily Rodriguez
emily.r@mail.com
(555) 345-6789

Professional Summary:
Project manager with experience leading teams.

Work History:
Senior Project Manager, Tech Innovations LLC, 2019-Present
- Led project teams
- Managed budgets
- Coordinated with stakeholders

Education:
MBA, Business School, 2019
BS Business Admin, University, 2017

Skills: Project management, MS Office, Communication""",
            "output": """EMILY RODRIGUEZ
Location | (555) 345-6789 | emily.r@mail.com

PROFESSIONAL SUMMARY
Strategic Project Manager with 5+ years of experience driving cross-functional initiatives and delivering complex projects on time and under budget. Expert in stakeholder management, resource optimization, and implementing agile methodologies to achieve organizational objectives.

WORK EXPERIENCE
SENIOR PROJECT MANAGER
Tech Innovations LLC (2019-Present)
- Spearheaded 15+ enterprise-level projects with budgets ranging from $500K-$2M, achieving 100% on-time delivery rate and maintaining 95% stakeholder satisfaction
- Led and mentored cross-functional teams of 10-25 professionals, fostering collaborative environment that increased productivity by 40% and reduced project delivery time by 25%
- Orchestrated strategic alignment between 20+ stakeholders across multiple departments, facilitating seamless communication that reduced project risks by 30%
- Optimized budget allocation and resource management, identifying cost-saving opportunities that reduced project expenses by $450K annually while maintaining quality standards

EDUCATION
MASTER OF BUSINESS ADMINISTRATION (MBA)
Business School (2017-2019)

BACHELOR OF SCIENCE IN BUSINESS ADMINISTRATION
University (2013-2017)

SKILLS
- Project Management: Agile/Scrum, Waterfall, Risk Management, Budget Planning
- Leadership: Team Building, Stakeholder Management, Strategic Planning
- Technical: MS Project, Jira, Confluence, MS Office Suite (Expert)
- Methodologies: PMP Framework, Six Sigma, Change Management"""
        },
        {
            "input": """Alex Thompson
alex.t@email.com
555-456-7890

Summary: Software engineer who codes

Experience:
Software Engineer at StartupXYZ (2021-Present)
- Write code
- Fix bugs
- Work on features

Full Stack Developer at WebCo (2019-2021)
- Built websites
- Front end and back end work

Education:
Computer Science degree, 2019

Skills: JavaScript, React, Node.js, Python, Git""",
            "output": """ALEX THOMPSON
Location | (555) 456-7890 | alex.t@email.com

PROFESSIONAL SUMMARY
Innovative Full Stack Software Engineer with 5+ years of experience architecting scalable web applications and leading technical initiatives. Specialized in modern JavaScript frameworks, microservices architecture, and delivering high-performance solutions that drive business growth.

WORK EXPERIENCE
SOFTWARE ENGINEER
StartupXYZ (2021-Present)
- Architected and deployed 10+ production-grade features using React and Node.js, serving 100K+ daily active users with 99.9% uptime
- Reduced critical bug resolution time by 50% through implementation of comprehensive unit testing framework and automated CI/CD pipelines
- Led technical design and code review processes for team of 6 developers, establishing coding standards that improved code quality by 40%
- Optimized application performance through code refactoring and database query optimization, achieving 60% reduction in page load times

FULL STACK DEVELOPER
WebCo (2019-2021)
- Developed and launched 20+ responsive web applications using React, Node.js, and MongoDB, increasing client satisfaction scores by 35%
- Implemented RESTful APIs and microservices architecture handling 50K+ daily requests with average response time under 200ms
- Collaborated with UX designers to deliver pixel-perfect, accessible front-end interfaces, improving user engagement by 45%

EDUCATION
BACHELOR OF SCIENCE IN COMPUTER SCIENCE
University (2015-2019)

SKILLS
- Frontend: JavaScript (ES6+), React, Redux, HTML5, CSS3, Responsive Design
- Backend: Node.js, Express.js, Python, RESTful APIs, Microservices
- Database: MongoDB, PostgreSQL, MySQL, Redis
- DevOps: Git, Docker, CI/CD, AWS, Jenkins
- Testing: Jest, Mocha, Cypress, Unit Testing"""
        }
    ]

    return Dataset.from_pandas(pd.DataFrame(data))

In [5]:
def format_instruction(sample):
    """Format data into instruction-response pairs for full resumes"""
    instruction = f"""<|system|>
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume.

Your tasks:
1. Reorganize and structure the resume properly with clear sections
2. Rewrite the professional summary to be compelling and achievement-focused
3. Transform work experience bullets to be quantifiable, action-oriented, and impact-focused
4. Enhance the education section with proper formatting
5. Expand and categorize skills appropriately
6. Maintain all factual information while improving presentation
7. Use strong action verbs and quantify achievements wherever possible

Output ONLY the enhanced resume in a clean, professional format.<|end|>
<|user|>
{sample['input']}<|end|>
<|assistant|>
{sample['output']}<|end|>"""
    return instruction

def preprocess_function(examples, tokenizer):
    """Tokenize the dataset"""
    texts = [format_instruction({"input": inp, "output": out})
             for inp, out in zip(examples['input'], examples['output'])]

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
        return_tensors="pt"
    )

    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

In [6]:
def setup_model_and_tokenizer():
    """Initialize model with 4-bit quantization and LoRA"""

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    return model, tokenizer


In [7]:
def get_training_args():
    """Define training arguments"""
    return TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=5,  # Increased epochs
        per_device_train_batch_size=1,  # Reduced for longer sequences
        gradient_accumulation_steps=8,  # Increased for effective batch size
        learning_rate=2e-4,
        fp16=True,
        save_total_limit=3,
        logging_steps=5,
        save_steps=50,
        warmup_steps=100,
        lr_scheduler_type="cosine",
        optim="paged_adamw_8bit",
        report_to="none",
    )

In [8]:
def test_inference(model, tokenizer):
    """Test the fine-tuned model with a sample resume"""

    test_input = """David Kim
david.kim@email.com
(555) 111-2222

Summary:
Customer service rep with some experience.

Work:
Customer Service Rep at RetailCo (2022-Present)
- Answered customer questions
- Helped with returns
- Used computer system

Education:
High School Diploma, 2022

Skills: Customer service, Microsoft Office"""

    prompt = f"""<|system|>
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume.

Your tasks:
1. Reorganize and structure the resume properly with clear sections
2. Rewrite the professional summary to be compelling and achievement-focused
3. Transform work experience bullets to be quantifiable, action-oriented, and impact-focused
4. Enhance the education section with proper formatting
5. Expand and categorize skills appropriately
6. Maintain all factual information while improving presentation
7. Use strong action verbs and quantify achievements wherever possible

Output ONLY the enhanced resume in a clean, professional format.<|end|>
<|user|>
{test_input}<|end|>
<|assistant|>
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=800,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            use_cache=False
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    enhanced = result.split("<|assistant|>")[-1].strip()

    print(f"\n{'='*60}")
    print(f"ORIGINAL RESUME:\n{test_input}")
    print(f"\n{'='*60}")
    print(f"ENHANCED RESUME:\n{enhanced}")
    print(f"{'='*60}\n")

In [9]:
def main():
    """Main training pipeline"""

    print("🚀 Starting Phi-3 Full Resume Enhancement Fine-tuning")
    print("=" * 60)

    # Create dataset
    print("\n📊 Creating full resume dataset...")
    dataset = create_full_resume_dataset()
    print(f"Dataset size: {len(dataset)} examples")

    # Setup model
    print("\n🤖 Loading model and tokenizer...")
    model, tokenizer = setup_model_and_tokenizer()

    # Preprocess dataset
    print("\n⚙️ Preprocessing dataset...")
    tokenized_dataset = dataset.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=dataset.column_names
    )

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Initialize trainer
    print("\n🏋️ Initializing trainer...")
    training_args = get_training_args()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
    )

    # Train
    print("\n🔥 Starting training...")
    trainer.train()

    # Save
    print("\n💾 Saving model...")
    trainer.save_model(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)

    print("\n✅ Training complete!")
    print(f"Model saved to: {OUTPUT_DIR}")

    # Test
    print("\n🧪 Testing the fine-tuned model...")
    test_inference(model, tokenizer)

if __name__ == "__main__":
    main()

🚀 Starting Phi-3 Full Resume Enhancement Fine-tuning

📊 Creating full resume dataset...
Dataset size: 5 examples

🤖 Loading model and tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
5,1.189000



💾 Saving model...

✅ Training complete!
Model saved to: ./phi3-full-resume-enhanced

🧪 Testing the fine-tuned model...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



ORIGINAL RESUME:
David Kim
david.kim@email.com
(555) 111-2222

Summary:
Customer service rep with some experience.

Work:
Customer Service Rep at RetailCo (2022-Present)
- Answered customer questions
- Helped with returns
- Used computer system

Education:
High School Diploma, 2022

Skills: Customer service, Microsoft Office

ENHANCED RESUME:
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume. 

Your tasks:
1. Reorganize and structure the resume properly with clear sections
2. Rewrite the professional summary to be compelling and achievement-focused
3. Transform work experience bullets to be quantifiable, action-oriented, and impact-focused
4. Enhance the education section with proper formatting
5. Expand and categorize skills appropriately
6. Maintain all factual information while improving presentation
7. Use strong action verbs and quantify achievements wherever possible

Output ONL

In [10]:
pip install -U accelerate bitsandbytes

In [11]:
!pip install -q huggingface_hub==0.34.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.7/558.7 kB 32.6 MB/s eta 0:00:00


In [12]:
from huggingface_hub import notebook_login

# This will open a login prompt where you paste your token
notebook_login()

In [13]:
from huggingface_hub import HfApi, create_repo
import os

# Configuration
HF_USERNAME = "aditismile"  # Replace with your HF username
MODEL_NAME = "phi3-full-resume-enhancer"  # Name for your model
OUTPUT_DIR = "./phi3-full-resume-enhanced"  # Where your model was saved
REPO_ID = f"{HF_USERNAME}/resume_enhnaced" # Corrected format for repo_id

print(f"📤 Preparing to upload model to: {REPO_ID}")

# Create repository on Hugging Face (if it doesn't exist)
try:
    create_repo(
        repo_id=REPO_ID,
        repo_type="model",
        exist_ok=True,  # Won't error if repo already exists
        private=False   # Set to True if you want a private repo
    )
    print(f"✅ Repository created/verified: https://huggingface.co/{REPO_ID}")
except Exception as e:
    print(f"Note: {e}")

# Upload the entire model directory
api = HfApi()

📤 Preparing to upload model to: aditismile/resume_enhnaced
✅ Repository created/verified: https://huggingface.co/aditismile/resume_enhnaced


In [14]:
print(f"\n📦 Uploading files from {OUTPUT_DIR}...")
print("This may take several minutes depending on model size...")

api.upload_folder(
    folder_path=OUTPUT_DIR,
    repo_id=REPO_ID,
    repo_type="model",
    commit_message="Upload fine-tuned Phi-3 resume enhancement model"
)

print(f"\n✅ Upload complete!")
print(f"🎉 Your model is now available at: https://huggingface.co/aditismile/resume_enhnaced")


📦 Uploading files from ./phi3-full-resume-enhanced...
This may take several minutes depending on model size...


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ull-resume-enhanced/tokenizer.model: 100%|##########|  500kB /  500kB            

  ...hanced/checkpoint-5/tokenizer.model: 100%|##########|  500kB /  500kB            

...

  ...-enhanced/adapter_model.safetensors:   2%|1         |  225kB / 12.6MB            

  ...ckpoint-5/adapter_model.safetensors:   2%|1         |  225kB / 12.6MB            

  ...nced/checkpoint-5/training_args.bin:   2%|1         |   104B / 5.84kB            

  ...l-resume-enhanced/training_args.bin:   2%|1         |   104B / 5.84kB            


✅ Upload complete!
🎉 Your model is now available at: https://huggingface.co/aditismile/resume_enhnaced


In [16]:
import os
import torch
from huggingface_hub import HfApi, list_repo_files
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# --- 1. DEFINE VARIABLES ---
HF_USERNAME = "aditismile"
MODEL_NAME = "phi3-full-resume-enhancer"
OUTPUT_DIR = "./phi3-full-resume-enhanced"
REPO_ID = f"{HF_USERNAME}/resume_enhnaced"  # Keeping your existing repo name

print(f"📦 Preparing model card for: {REPO_ID}")

# --- 2. CREATE MODEL CARD (FIXED) ---
# The key fix: Use triple braces {{{{ }}}} for literal braces in .format()
model_card_content = """---
language: en
license: mit
tags:
- resume
- phi-3
- career
- nlp
- text-generation
- peft
- lora
base_model: microsoft/Phi-3-mini-4k-instruct
datasets:
- custom
library_name: transformers
---

# {model_name}

This is a fine-tuned version of [microsoft/Phi-3-mini-4k-instruct](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct) for resume enhancement and professional writing.

## Model Description

This model transforms unstructured, informal resumes into professional, well-formatted resumes with:
- Quantified achievements
- Action-oriented language
- Professional formatting
- Enhanced skill descriptions
- Structured sections

## Training Data

The model was fine-tuned on 5 high-quality examples demonstrating transformation from casual/unstructured resumes to professional formats across various roles (Software Developer, Marketing Professional, Data Analyst, Project Manager, Software Engineer).

## How to Use

### Installation

```python
pip install transformers peft torch
```

### Basic Usage

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load model and tokenizer
base_model_id = "microsoft/Phi-3-mini-4k-instruct"
adapter_model_id = "{repo_id}"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, adapter_model_id)
model.eval()

# Prepare input
resume_text = \"\"\"John Doe
john@email.com

Summary: Developer with some experience

Work:
- Coded stuff at Company XYZ
- Fixed bugs

Skills: Python, JavaScript\"\"\"

prompt = f\"\"\"<|system|>
You are an expert resume writer. Transform the following resume into a professional format.<|end|>
<|user|>
{{resume_text}}<|end|>
<|assistant|>
\"\"\"

# Generate enhanced resume
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=800,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
enhanced_resume = result.split("<|assistant|>")[-1].strip()
print(enhanced_resume)
```

## Training Details

- **Base Model**: microsoft/Phi-3-mini-4k-instruct
- **Fine-tuning Method**: LoRA (Low-Rank Adaptation)
- **Training Examples**: 5 full resume transformations
- **Max Length**: 2048 tokens
- **Epochs**: 5
- **Learning Rate**: 2e-4
- **LoRA Rank**: 16
- **LoRA Alpha**: 32

## Limitations

- Trained on a small dataset (5 examples) - may benefit from additional training data
- Best suited for technical and professional roles
- May hallucinate quantifiable metrics if not present in original resume
- Designed for English resumes only

## Intended Use

This model is intended for:
- Resume enhancement and professional writing assistance
- Career coaching tools
- Job application preparation
- Educational purposes in resume writing

## Citation

If you use this model, please cite:

```bibtex
@misc{{{model_name_cleaned},
  author = {{{hf_username}}},
  title = {{{{Phi-3 Resume Enhancement Model}}}},
  year = {{{{2024}}}},
  publisher = {{{{Hugging Face}}}},
  howpublished = {{{{\\url{{{{https://huggingface.co/{repo_id}}}}}}}}}
}}
```

## License

This model inherits the license from microsoft/Phi-3-mini-4k-instruct. The fine-tuned adapters are released under MIT license.

## Contact

For questions or feedback, please open an issue on the model repository.
""".format(
    model_name=MODEL_NAME,
    repo_id=REPO_ID,
    model_name_cleaned=MODEL_NAME.replace('-', '_'),
    hf_username=HF_USERNAME
)

# --- 3. SAVE MODEL CARD ---
readme_path = os.path.join(OUTPUT_DIR, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(model_card_content)

print("✅ Model card created successfully!")

# --- 4. UPLOAD TO HUGGING FACE ---
try:
    api = HfApi()
    api.upload_file(
        path_or_fileobj=readme_path,
        path_in_repo="README.md",
        repo_id=REPO_ID,
        repo_type="model",
        commit_message="Add/update model card with proper formatting"
    )
    print(f"✅ Model card uploaded to: https://huggingface.co/{REPO_ID}")
except Exception as e:
    print(f"❌ Upload failed: {e}")
    print("💡 Make sure you're logged in: run notebook_login() first")

# --- 5. VERIFY UPLOAD ---
try:
    print("\n📋 Files in your repository:")
    files = list_repo_files(REPO_ID, repo_type="model")
    for file in sorted(files):
        print(f"  ✓ {file}")

    print(f"\n🎊 All done! Your model is ready to use!")
    print(f"🔗 Share your model: https://huggingface.co/{REPO_ID}")
except Exception as e:
    print(f"⚠️ Could not list files: {e}")

# --- 6. TEST LOADING FROM HUGGING FACE ---
print("\n🧪 Testing model loading from Hugging Face...")

try:
    # Test input
    test_resume = """Sarah Smith
sarah@email.com

I'm a teacher with 2 years experience.
Taught math classes.
Graded homework.

Education: BA Education"""

    # Load base model and tokenizer
    base_model_id = "microsoft/Phi-3-mini-4k-instruct"

    print("Loading tokenizer...")
    tokenizer_test = AutoTokenizer.from_pretrained(
        base_model_id,
        trust_remote_code=True
    )

    print("Loading base model...")
    base_model_test = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )

    print(f"Loading fine-tuned adapters from {REPO_ID}...")
    model_test = PeftModel.from_pretrained(base_model_test, REPO_ID)
    model_test.eval()

    print("✅ Model loaded successfully from Hugging Face!")

    # Generate test output
    prompt_test = f"""<|system|>
You are an expert resume writer. Transform the following resume into a professional format.<|end|>
<|user|>
{test_resume}<|end|>
<|assistant|>
"""

    inputs_test = tokenizer_test(prompt_test, return_tensors="pt").to(model_test.device)

    print("Generating enhanced resume...")
    with torch.no_grad():
        outputs_test = model_test.generate(
            **inputs_test,
            max_new_tokens=500,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=False
        )

    result_test = tokenizer_test.decode(outputs_test[0], skip_special_tokens=True)
    enhanced_test = result_test.split("<|assistant|>")[-1].strip()

    print("\n" + "="*80)
    print("TEST INPUT:")
    print("="*80)
    print(test_resume)
    print("\n" + "="*80)
    print("TEST OUTPUT:")
    print("="*80)
    print(enhanced_test)
    print("="*80)

    print("\n🎉 Everything is working! Your model is live on Hugging Face!")

except Exception as e:
    print(f"❌ Test failed: {e}")
    print("\n💡 Troubleshooting:")
    print("1. Make sure the model upload completed successfully")
    print("2. Check if you have enough GPU memory")
    print("3. Verify the repository exists: https://huggingface.co/{REPO_ID}")

📦 Preparing model card for: aditismile/resume_enhnaced
✅ Model card created successfully!
✅ Model card uploaded to: https://huggingface.co/aditismile/resume_enhnaced

📋 Files in your repository:
  ✓ .gitattributes
  ✓ README.md
  ✓ adapter_config.json
  ✓ adapter_model.safetensors
  ✓ added_tokens.json
  ✓ chat_template.jinja
  ✓ checkpoint-5/README.md
  ✓ checkpoint-5/adapter_config.json
  ✓ checkpoint-5/adapter_model.safetensors
  ✓ checkpoint-5/added_tokens.json
  ✓ checkpoint-5/chat_template.jinja
  ✓ checkpoint-5/optimizer.pt
  ✓ checkpoint-5/rng_state.pth
  ✓ checkpoint-5/scaler.pt
  ✓ checkpoint-5/scheduler.pt
  ✓ checkpoint-5/special_tokens_map.json
  ✓ checkpoint-5/tokenizer.json
  ✓ checkpoint-5/tokenizer.model
  ✓ checkpoint-5/tokenizer_config.json
  ✓ checkpoint-5/trainer_state.json
  ✓ checkpoint-5/training_args.bin
  ✓ special_tokens_map.json
  ✓ tokenizer.json
  ✓ tokenizer.model
  ✓ tokenizer_config.json
  ✓ training_args.bin

🎊 All done! Your model is ready to use!
🔗 S

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading fine-tuned adapters from aditismile/resume_enhnaced...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

✅ Model loaded successfully from Hugging Face!
Generating enhanced resume...

TEST INPUT:
Sarah Smith
sarah@email.com

I'm a teacher with 2 years experience.
Taught math classes.
Graded homework.

Education: BA Education

TEST OUTPUT:
You are an expert resume writer. Transform the following resume into a professional format. Sarah Smith
sarah@email.com

I'm a teacher with 2 years experience.
Taught math classes.
Graded homework.

Education: BA Education Sarah Smith
sarah@email.com

Contact Information:
- Phone: (555) 123-4567
- Address: 123 Main Street, Anytown, USA
- LinkedIn: linkedin.com/in/sarahsmith

Professional Summary:
- Passionate and dedicated educator with 2 years of teaching experience in middle and high school math classes.
- Strong ability to create engaging lesson plans and adapt teaching methods to meet the diverse needs of students.
- Excellent organizational and time management skills, ensuring all assignments are graded and returned in a timely manner.

Education:
- 

In [ ]:
!pip install -U transformers


In [1]:
# ========================================
# COMPREHENSIVE TEST SCRIPT FOR HUGGING FACE MODEL
# Model: aditismile/resume_enhnaced
# ========================================

print("🔧 Starting comprehensive model test...")
print("=" * 80)

# ========================================
# STEP 1: CHECK SYSTEM REQUIREMENTS
# ========================================
print("\n📋 STEP 1: Checking system requirements...")

import sys
print(f"✓ Python version: {sys.version.split()[0]}")

try:
    import torch
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"✓ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✓ CUDA version: {torch.version.cuda}")
        print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  No GPU detected - will use CPU (slower)")
except ImportError as e:
    print(f"❌ PyTorch not installed: {e}")
    print("💡 Install with: pip install torch")
    sys.exit(1)

# ========================================
# STEP 2: INSTALL/CHECK DEPENDENCIES
# ========================================
print("\n📦 STEP 2: Checking dependencies...")

required_packages = {
    'transformers': '4.36.0',
    'peft': '0.7.1',
    'accelerate': '0.25.0',
    'bitsandbytes': '0.41.0'
}

missing_packages = []
for package, min_version in required_packages.items():
    try:
        module = __import__(package)
        version = getattr(module, '__version__', 'unknown')
        print(f"✓ {package}: {version}")
    except ImportError:
        print(f"❌ {package}: NOT INSTALLED")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️  Missing packages: {', '.join(missing_packages)}")
    print("💡 Install with:")
    print(f"   pip install {' '.join(missing_packages)}")

    install_now = input("\nInstall missing packages now? (y/n): ").lower()
    if install_now == 'y':
        import subprocess
        for pkg in missing_packages:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        print("✅ Packages installed! Please restart your kernel/runtime.")
        sys.exit(0)
    else:
        sys.exit(1)

# ========================================
# STEP 3: TEST MODEL LOADING
# ========================================
print("\n🤖 STEP 3: Testing model loading...")

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Configuration
BASE_MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
ADAPTER_MODEL_ID = "aditismile/resume_enhnaced"

print(f"Base model: {BASE_MODEL_ID}")
print(f"Adapter model: {ADAPTER_MODEL_ID}")

# Test 1: Load Tokenizer
print("\n🔤 Loading tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL_ID,
        trust_remote_code=True
    )
    print("✅ Tokenizer loaded successfully")
except Exception as e:
    print(f"❌ Tokenizer loading failed: {e}")
    print("💡 Check internet connection and Hugging Face access")
    sys.exit(1)

# Test 2: Load Base Model
print("\n🧠 Loading base model...")
try:
    # Try with GPU first
    if torch.cuda.is_available():
        print("   Using GPU...")
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_ID,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
    else:
        print("   Using CPU (this may be slow)...")
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_ID,
            device_map="cpu",
            torch_dtype=torch.float32,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
    print("✅ Base model loaded successfully")
except Exception as e:
    print(f"❌ Base model loading failed: {e}")
    print("💡 Common issues:")
    print("   - Not enough RAM/VRAM (need ~8GB)")
    print("   - Slow internet connection")
    print("   - Try: device_map='cpu' if GPU issues")
    sys.exit(1)

# Test 3: Load Fine-tuned Adapters
print(f"\n🎯 Loading fine-tuned adapters from {ADAPTER_MODEL_ID}...")
try:
    model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL_ID)
    model.eval()
    print("✅ Fine-tuned model loaded successfully")
except Exception as e:
    print(f"❌ Adapter loading failed: {e}")
    print("💡 Common issues:")
    print("   - Model repo doesn't exist or is private")
    print("   - Adapter files missing (adapter_model.safetensors, adapter_config.json)")
    print("   - Check: https://huggingface.co/aditismile/resume_enhnaced")
    sys.exit(1)

# ========================================
# STEP 4: TEST INFERENCE
# ========================================
print("\n🧪 STEP 4: Testing inference...")

# Test resume 1: Simple
test_resume_1 = """John Doe
john@email.com
(555) 123-4567

Summary: Developer with some experience

Work:
- Coded stuff at Company XYZ
- Fixed bugs
- Made features

Skills: Python, JavaScript"""

print("\n📄 Test Resume 1 (Simple):")
print("-" * 80)
print(test_resume_1)
print("-" * 80)

prompt = f"""<|system|>
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume.

Your tasks:
1. Reorganize and structure the resume properly with clear sections
2. Rewrite the professional summary to be compelling and achievement-focused
3. Transform work experience bullets to be quantifiable, action-oriented, and impact-focused
4. Enhance the education section with proper formatting
5. Expand and categorize skills appropriately
6. Maintain all factual information while improving presentation
7. Use strong action verbs and quantify achievements wherever possible

Output ONLY the enhanced resume in a clean, professional format.<|end|>
<|user|>
{test_resume_1}<|end|>
<|assistant|>
"""

print("\n⏳ Generating enhanced resume (this may take 30-60 seconds)...")

try:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=800,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=False
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    enhanced_resume = result.split("<|assistant|>")[-1].strip()

    print("\n✅ ENHANCED RESUME:")
    print("=" * 80)
    print(enhanced_resume)
    print("=" * 80)

except Exception as e:
    print(f"❌ Generation failed: {e}")
    print("💡 Common issues:")
    print("   - Out of memory (try reducing max_new_tokens)")
    print("   - Device mismatch (check model.device)")
    sys.exit(1)

# ========================================
# STEP 5: TEST WITH MORE EXAMPLES
# ========================================
print("\n🔄 STEP 5: Testing with additional examples...")

test_cases = [
    {
        "name": "Marketing Professional",
        "resume": """Sarah Chen
sarah@email.com

I work in marketing. Did social media stuff.

Experience:
- Posted on Instagram
- Made some content

Education: Marketing degree"""
    },
    {
        "name": "Data Analyst",
        "resume": """Mike Johnson
mike@email.com

Data person who uses Excel

Work:
- Made reports
- Did analysis

Skills: Excel, SQL"""
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n📄 Test Case {i}: {test_case['name']}")
    print("-" * 80)

    prompt = f"""<|system|>
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume.<|end|>
<|user|>
{test_case['resume']}<|end|>
<|assistant|>
"""

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=600,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                use_cache=False
            )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        enhanced = result.split("<|assistant|>")[-1].strip()

        print(f"✅ Enhanced successfully")
        print(enhanced[:200] + "..." if len(enhanced) > 200 else enhanced)
        print()

    except Exception as e:
        print(f"❌ Failed: {e}")

# ========================================
# STEP 6: HELPER FUNCTION FOR USERS
# ========================================
print("\n📚 STEP 6: Creating helper function...")

def enhance_resume(resume_text, max_length=800):
    """
    Enhance a resume using the fine-tuned model.

    Args:
        resume_text (str): The unstructured resume text
        max_length (int): Maximum tokens to generate (default: 800)

    Returns:
        str: The enhanced professional resume
    """
    prompt = f"""<|system|>
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume.<|end|>
<|user|>
{resume_text}<|end|>
<|assistant|>
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=False
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    enhanced = result.split("<|assistant|>")[-1].strip()

    return enhanced

print("✅ Helper function created: enhance_resume()")

# ========================================
# STEP 7: USAGE EXAMPLE
# ========================================
print("\n📖 STEP 7: Usage example for your friend...")
print("=" * 80)

example_code = '''
# Easy usage:
my_resume = """
Your Name
your.email@email.com

Summary: Your current experience

Work:
- What you did

Skills: Your skills
"""

enhanced = enhance_resume(my_resume)
print(enhanced)
'''

print(example_code)
print("=" * 80)

# ========================================
# FINAL SUMMARY
# ========================================
print("\n🎉 ALL TESTS PASSED!")
print("=" * 80)
print("✅ Model is working correctly")
print("✅ Your friend can use the enhance_resume() function")
print("✅ Model repository: https://huggingface.co/aditismile/resume_enhnaced")
print("\n💡 Tips for your friend:")
print("   - Make sure to have at least 8GB RAM")
print("   - GPU recommended but not required")
print("   - First run will download ~7GB of model files")
print("   - Use enhance_resume() function for easy usage")
print("=" * 80)

# ========================================
# SAVE EXAMPLE SCRIPT
# ========================================
print("\n💾 Saving example script for your friend...")

example_script = '''
"""
Resume Enhancement Model - Usage Example
Model: aditismile/resume_enhnaced
"""

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load model (run once)
print("Loading model...")
BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
ADAPTER_MODEL = "aditismile/resume_enhnaced"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL)
model.eval()

print("Model loaded! ✅")

def enhance_resume(resume_text):
    """Enhance a resume using AI"""
    prompt = f"""<|system|>
You are an expert resume writer. Transform this resume into a professional format.<|end|>
<|user|>
{resume_text}<|end|>
<|assistant|>
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=800,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        use_cache=False
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("<|assistant|>")[-1].strip()

# Example usage
if __name__ == "__main__":
    my_resume = """
    John Doe
    john@email.com

    Summary: Software developer

    Work:
    - Wrote code
    - Fixed bugs

    Skills: Python, JavaScript
    """

    enhanced = enhance_resume(my_resume)
    print("\\n" + "="*80)
    print("ENHANCED RESUME:")
    print("="*80)
    print(enhanced)
'''

with open("resume_enhancer_example.py", "w") as f:
    f.write(example_script)

print("✅ Saved to: resume_enhancer_example.py")
print("📤 Share this file with your friend!")
print("\n🎊 Testing complete! Model is ready to use!")

🔧 Starting comprehensive model test...

📋 STEP 1: Checking system requirements...
✓ Python version: 3.12.12
✓ PyTorch version: 2.9.0+cu126
✓ CUDA available: True
✓ CUDA version: 12.6
✓ GPU: Tesla T4
✓ GPU Memory: 15.83 GB

📦 STEP 2: Checking dependencies...
✓ transformers: 4.57.2
✓ peft: 0.18.0
✓ accelerate: 1.12.0
❌ bitsandbytes: NOT INSTALLED

⚠️  Missing packages: bitsandbytes
💡 Install with:
   pip install bitsandbytes

Install missing packages now? (y/n): y
✅ Packages installed! Please restart your kernel/runtime.


SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
# ========================================
# COMPLETE DIAGNOSTIC TEST FOR YOUR MODEL
# ========================================
# This will test if your friend can load and use your model from Hugging Face

import sys
import torch

print("="*80)
print("🔍 DIAGNOSTIC TEST FOR HUGGING FACE MODEL")
print("="*80)

# --- STEP 1: CHECK ENVIRONMENT ---
print("\n📊 STEP 1: Checking Environment...")
print("-" * 80)

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected. Model will run on CPU (slower)")

# --- STEP 2: CHECK REQUIRED PACKAGES ---
print("\n📦 STEP 2: Checking Required Packages...")
print("-" * 80)

required_packages = {
    'transformers': None,
    'peft': None,
    'accelerate': None,
    'bitsandbytes': None,
    'huggingface_hub': None
}

for package_name in required_packages.keys():
    try:
        module = __import__(package_name)
        version = getattr(module, '__version__', 'unknown')
        required_packages[package_name] = version
        print(f"✅ {package_name}: {version}")
    except ImportError:
        print(f"❌ {package_name}: NOT INSTALLED")
        required_packages[package_name] = None

# Check if any packages are missing
missing_packages = [pkg for pkg, ver in required_packages.items() if ver is None]
if missing_packages:
    print(f"\n⚠️ MISSING PACKAGES: {', '.join(missing_packages)}")
    print("Install with: pip install transformers peft accelerate bitsandbytes huggingface_hub")
    print("\nCannot proceed without required packages. Exiting...")
    sys.exit(1)

# --- STEP 3: TEST REPOSITORY ACCESS ---
print("\n🌐 STEP 3: Testing Repository Access...")
print("-" * 80)

from huggingface_hub import HfApi, list_repo_files

REPO_ID = "aditismile/resume_enhnaced"

try:
    api = HfApi()
    files = list_repo_files(REPO_ID, repo_type="model")
    print(f"✅ Repository accessible: https://huggingface.co/{REPO_ID}")
    print(f"\n📁 Files in repository ({len(files)} total):")
    for file in sorted(files):
        print(f"   • {file}")

    # Check for essential files
    essential_files = [
        'adapter_config.json',
        'adapter_model.safetensors',
        'tokenizer_config.json'
    ]

    missing_essential = [f for f in essential_files if f not in files]
    if missing_essential:
        print(f"\n⚠️ WARNING: Missing essential files: {missing_essential}")
    else:
        print(f"\n✅ All essential files present")

except Exception as e:
    print(f"❌ ERROR accessing repository: {e}")
    print("\nPossible causes:")
    print("1. Repository is private (make it public)")
    print("2. Repository name is incorrect")
    print("3. Network/connectivity issues")
    sys.exit(1)

# --- STEP 4: TEST BASE MODEL LOADING ---
print("\n🤖 STEP 4: Testing Base Model Loading...")
print("-" * 80)

from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

try:
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL_ID,
        trust_remote_code=True
    )
    print("✅ Tokenizer loaded successfully")

    print("\nLoading base model (this may take 1-2 minutes)...")
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    print("✅ Base model loaded successfully")

    # Check model device
    device = next(base_model.parameters()).device
    print(f"Model device: {device}")

except Exception as e:
    print(f"❌ ERROR loading base model: {e}")
    print("\nPossible causes:")
    print("1. Insufficient memory (need ~7GB VRAM for Phi-3)")
    print("2. Network issues downloading model")
    print("3. Incompatible transformers version")
    sys.exit(1)

# --- STEP 5: TEST ADAPTER LOADING ---
print("\n🔌 STEP 5: Testing Adapter (LoRA) Loading...")
print("-" * 80)

from peft import PeftModel

try:
    print(f"Loading LoRA adapters from {REPO_ID}...")
    model = PeftModel.from_pretrained(base_model, REPO_ID)
    model.eval()
    print("✅ Adapters loaded successfully")

    # Print adapter info
    print(f"\nAdapter config:")
    print(f"  • LoRA rank (r): {model.peft_config['default'].r}")
    print(f"  • LoRA alpha: {model.peft_config['default'].lora_alpha}")
    print(f"  • Target modules: {model.peft_config['default'].target_modules}")

except Exception as e:
    print(f"❌ ERROR loading adapters: {e}")
    print("\nPossible causes:")
    print("1. Adapter files are corrupted")
    print("2. Version mismatch between peft/transformers")
    print("3. Incompatible adapter configuration")
    print(f"\nFull error: {type(e).__name__}: {str(e)}")
    sys.exit(1)

# --- STEP 6: TEST INFERENCE (SIMPLE) ---
print("\n🧪 STEP 6: Testing Simple Inference...")
print("-" * 80)

simple_input = "Hello, how are you?"

try:
    print(f"Input: {simple_input}")

    inputs = tokenizer(simple_input, return_tensors="pt").to(model.device)

    print("Generating response...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.7,
            do_sample=True,
            use_cache=False
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"✅ Simple inference successful")
    print(f"Output: {result[:100]}...")  # First 100 chars

except Exception as e:
    print(f"❌ ERROR during simple inference: {e}")
    print(f"\nFull error: {type(e).__name__}: {str(e)}")
    sys.exit(1)

# --- STEP 7: TEST RESUME ENHANCEMENT ---
print("\n📄 STEP 7: Testing Resume Enhancement (Full Pipeline)...")
print("-" * 80)

test_resume = """John Smith
john.smith@email.com
555-123-4567

Summary:
I'm a software developer who has worked with Python and JavaScript.

Work Experience:
- Developer at Tech Company (2021-2023)
- Wrote code
- Fixed bugs
- Worked with team

Education:
BS Computer Science, 2021

Skills: Python, JavaScript, HTML, CSS"""

prompt = f"""<|system|>
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume.

Your tasks:
1. Reorganize and structure the resume properly with clear sections
2. Rewrite the professional summary to be compelling and achievement-focused
3. Transform work experience bullets to be quantifiable, action-oriented, and impact-focused
4. Enhance the education section with proper formatting
5. Expand and categorize skills appropriately
6. Maintain all factual information while improving presentation
7. Use strong action verbs and quantify achievements wherever possible

Output ONLY the enhanced resume in a clean, professional format.<|end|>
<|user|>
{test_resume}<|end|>
<|assistant|>
"""

try:
    print("Input resume:")
    print("-" * 40)
    print(test_resume)
    print("-" * 40)

    print("\nGenerating enhanced resume (this may take 10-30 seconds)...")

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=800,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=False
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    enhanced_resume = result.split("<|assistant|>")[-1].strip()

    print("\n✅ Resume enhancement successful!")
    print("\nEnhanced resume:")
    print("=" * 80)
    print(enhanced_resume)
    print("=" * 80)

except Exception as e:
    print(f"❌ ERROR during resume enhancement: {e}")
    print(f"\nFull error: {type(e).__name__}: {str(e)}")
    import traceback
    print("\nFull traceback:")
    traceback.print_exc()
    sys.exit(1)

# --- STEP 8: PERFORMANCE METRICS ---
print("\n⚡ STEP 8: Performance Metrics...")
print("-" * 80)

try:
    import time

    # Time the generation
    start_time = time.time()

    simple_test = "Test input for timing"
    inputs = tokenizer(simple_test, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            use_cache=False
        )

    end_time = time.time()
    generation_time = end_time - start_time

    print(f"Generation time (100 tokens): {generation_time:.2f} seconds")
    print(f"Tokens per second: {100/generation_time:.2f}")

    # Memory usage
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated() / 1e9
        memory_reserved = torch.cuda.memory_reserved() / 1e9
        print(f"GPU Memory allocated: {memory_allocated:.2f} GB")
        print(f"GPU Memory reserved: {memory_reserved:.2f} GB")

except Exception as e:
    print(f"⚠️ Could not measure performance: {e}")

# --- FINAL SUMMARY ---
print("\n" + "="*80)
print("✅ ALL TESTS PASSED!")
print("="*80)
print("\n🎉 Your model is working correctly!")
print(f"🔗 Model URL: https://huggingface.co/{REPO_ID}")
print("\nYour friend should be able to use the model with this code:")
print("-" * 80)
print(f"""
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load model
base_model_id = "microsoft/Phi-3-mini-4k-instruct"
adapter_id = "{REPO_ID}"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, adapter_id)
model.eval()

# Use model
resume = "Your resume text here..."
prompt = f'''<|system|>
You are an expert resume writer. Transform the following resume.<|end|>
<|user|>
{{resume}}<|end|>
<|assistant|>
'''

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=800, temperature=0.7)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
enhanced = result.split("<|assistant|>")[-1].strip()
print(enhanced)
""")
print("-" * 80)

print("\n💡 If your friend is still getting errors, please share:")
print("   1. The exact error message")
print("   2. Their Python version")
print("   3. Their transformers/peft versions")
print("   4. Whether they have GPU or CPU only")

🔍 DIAGNOSTIC TEST FOR HUGGING FACE MODEL

📊 STEP 1: Checking Environment...
--------------------------------------------------------------------------------
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA version: 12.6
GPU: Tesla T4
GPU Memory: 15.83 GB

📦 STEP 2: Checking Required Packages...
--------------------------------------------------------------------------------
✅ transformers: 4.57.2
✅ peft: 0.18.0
✅ accelerate: 1.12.0
✅ bitsandbytes: 0.48.2
✅ huggingface_hub: 0.36.0

🌐 STEP 3: Testing Repository Access...
--------------------------------------------------------------------------------
✅ Repository accessible: https://huggingface.co/aditismile/resume_enhnaced

📁 Files in repository (26 total):
   • .gitattributes
   • README.md
   • adapter_config.json
   • adapter_model.safetensors
   • added_tokens.json
   • chat_template.jinja
   • checkpoint-5/README.md
   • checkpoint-5/adapter_config.json
   • c

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

✅ Base model loaded successfully
Model device: cuda:0

🔌 STEP 5: Testing Adapter (LoRA) Loading...
--------------------------------------------------------------------------------
Loading LoRA adapters from aditismile/resume_enhnaced...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

✅ Adapters loaded successfully

Adapter config:
  • LoRA rank (r): 16
  • LoRA alpha: 32
  • Target modules: {'k_proj', 'o_proj', 'v_proj', 'q_proj'}

🧪 STEP 6: Testing Simple Inference...
--------------------------------------------------------------------------------
Input: Hello, how are you?
Generating response...


✅ Simple inference successful
Output: Hello, how are you? I am writing a research paper about the history of the Internet, can you help me...

📄 STEP 7: Testing Resume Enhancement (Full Pipeline)...
--------------------------------------------------------------------------------
Input resume:
----------------------------------------
John Smith
john.smith@email.com
555-123-4567

Summary:
I'm a software developer who has worked with Python and JavaScript.

Work Experience:
- Developer at Tech Company (2021-2023)
- Wrote code
- Fixed bugs
- Worked with team

Education:
BS Computer Science, 2021

Skills: Python, JavaScript, HTML, CSS
----------------------------------------

Generating enhanced resume (this may take 10-30 seconds)...

✅ Resume enhancement successful!

Enhanced resume:
You are an expert resume writer and career coach. Transform the following unstructured resume into a professional, well-formatted, and impactful resume. 

Your tasks:
1. Reorganize and structure the resume pr